In [2]:
import json
from glob import glob

In [5]:
# CONFIG = 'hopper_custom_rl' 
# DATASET = 'H-MIXED-RT-4'
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Hopper_H-MIXED-RT-4/*/*"
# rollout_lengths = {1, 5, 50}

CONFIG = 'halfcheetah_custom_rl' 
DATASET = 'MIXED-RT-5'
model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/*/*"
rollout_lengths = {5, 20} #{1, 5, 20}

# CONFIG = 'walker2d_custom_rl' 
# DATASET = 'W-MIXED-RT-1' 
# model_dir_glob = "/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/Walker2d_W-MIXED-RT-1/*/*"
# rollout_lengths = {1, 5, 10}

mopo_penalty_coeffs = {0, 1, 5}
seeds = {1, 2, 3}
# seeds = {1,}

In [7]:
dir_list = glob(model_dir_glob, recursive = True)

def filter_dir(x):
    last = x.split('/')[-1]
    return last.startswith('s') 

with_s = list(filter(filter_dir, dir_list))
with_s

['/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexb20/s6_2023-03-13_21-16-515m1m92i9',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexstdb01/s6_2023-03-13_21-16-51zp5dtmoo',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexmeandevb1/s6_2023-03-13_21-16-51dgn3nklk',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexmeandevb05/s6_2023-03-13_21-16-51g8mukbca',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexstdb8/s6_2023-03-13_21-16-51ogf7xx5d',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexmeandevb2/s6_2023-03-13_21-16-51toyusw1h',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexmeandevb01/s6_2023-03-13_21-16-51rjfrtgk2',
 '/rds/project/rds-eWkDxBhxBrQ/dimorl/code/mopo/ray_mopo/HalfCheetah_MIXED-RT-1/rexstdb1/s6_2023-03-13_21-16-51_foi_owr',
 '/rds/

In [8]:
dir_list = glob(model_dir_glob, recursive = True)

def filter_dir(x):
    last = x.split('/')[-1]
    return last.startswith('s') 

with_s = list(filter(filter_dir, dir_list))
with_s = ["/".join(model_dir.split('/')[-3:])for model_dir in with_s]
model_load_dirs = with_s
with_s

['HalfCheetah_MIXED-RT-1/rexb20/s6_2023-03-13_21-16-515m1m92i9',
 'HalfCheetah_MIXED-RT-1/rexstdb01/s6_2023-03-13_21-16-51zp5dtmoo',
 'HalfCheetah_MIXED-RT-1/rexmeandevb1/s6_2023-03-13_21-16-51dgn3nklk',
 'HalfCheetah_MIXED-RT-1/rexmeandevb05/s6_2023-03-13_21-16-51g8mukbca',
 'HalfCheetah_MIXED-RT-1/rexstdb8/s6_2023-03-13_21-16-51ogf7xx5d',
 'HalfCheetah_MIXED-RT-1/rexmeandevb2/s6_2023-03-13_21-16-51toyusw1h',
 'HalfCheetah_MIXED-RT-1/rexmeandevb01/s6_2023-03-13_21-16-51rjfrtgk2',
 'HalfCheetah_MIXED-RT-1/rexstdb1/s6_2023-03-13_21-16-51_foi_owr',
 'HalfCheetah_MIXED-RT-1/rexb10/s6_2023-03-13_21-16-51ciug6ocr',
 'HalfCheetah_MIXED-RT-1/rexstdb05/s6_2023-03-13_21-16-51gobcvtlx',
 'HalfCheetah_MIXED-RT-1/rexb5/s6_2023-03-13_21-16-513xyv2dwt',
 'HalfCheetah_MIXED-RT-1/default/s6_2023-03-13_21-08-25cjd50s30',
 'HalfCheetah_MIXED-RT-1/default/s6_2023-03-13_21-16-51vg2vhn8p',
 'HalfCheetah_MIXED-RT-1/rexstdb2/s6_2023-03-13_21-16-51rvwza6li']

In [9]:
# model_load_dirs = [mld for mld in  model_load_dirs if '_5_' in mld]
# model_load_dirs

In [10]:
# model_load_dirs = ['HalfCheetah_MIXED-RT-5/default/s1_2023-03-06_11-13-45xh8hsuja']

In [11]:
list_to_dump = []

for mld in model_load_dirs:
    bnn_name = mld.split('/')[-2]
    bnn_seed = mld.split('/')[-1].split('_')[0]
    for rl in rollout_lengths:
        for mp in mopo_penalty_coeffs:
            for seed in seeds:
                exp_name = f'{bnn_name}{bnn_seed}_mp{mp}rl{rl}'
                list_to_dump.append(dict(config=CONFIG, dataset=DATASET, rollout_length=rl, mopo_penalty_coeff=mp,
                                    seed=seed, model_load_dir=mld, exp_name=exp_name)) 

with open(f"{DATASET}_exp_params.json", "w") as outfile:
    json.dump(list_to_dump, outfile, indent="")

In [12]:
f"{DATASET}_exp_params.json"

'MIXED-RT-5_exp_params.json'